In [17]:
"""
Created on Tue 2023-10-10
Last updated Tue 2023-12-05
version: 1.3
author: David France
email: david.france@snhu.edu
Purpose: Provide Dash App dashboard for displaying Austin Animal
         Shelter database information.
"""

# Setup the Jupyter version of Dash
from dash import Dash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table, ctx
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template
from datetime import date
from geopy.geocoders import Nominatim
import time
import base64
from datetime import datetime

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import CRUD module
from animalshelter import AnimalShelter
import dash_css 

# Connect to databases
db = AnimalShelter("AAC", "animals")
db_backup = AnimalShelter("AAC_Backup", "animals")
db_users = AnimalShelter("AAC", "users")

css = dash_css.CSS()

# Populate data table with all records from AnimalShelter
df = pd.DataFrame.from_records(db.find({}))

#########################
# Input/Filter Layouts
#########################

# Filters for sorting datatable view
dropdown_filters = [html.Div(
                    dcc.Dropdown(df.animal_type.unique(), multi=True, searchable=True, placeholder='Animal types', id='animal_select'),
                    style={'width': '15%', 'display': 'inline-block'}),
                  html.Div(
                    dcc.Dropdown(df.breed.unique(), multi=True, searchable=True, placeholder='Breeds', id='breed_select'),
                    style={'width': '25%', 'display': 'inline-block'}),
                  html.Div(
                    dcc.Dropdown(df.color.unique(), multi=True, searchable=True, placeholder='Color', id='color_select'),
                    style={'width': '20%', 'display': 'inline-block'}),
                  html.Div(
                    dcc.Dropdown(df.sex_upon_outcome.unique(), multi=True, searchable=True, placeholder='Sex on outcome', id='sex_select'),
                    style={'width': '20%', 'display': 'inline-block'}),
                  html.Div(
                      dcc.Dropdown(options=[{"label":str(i),"value":i} for i in range(0,1501)], 
                                   searchable=True, 
                                   placeholder='Min age weeks', 
                                   id='range-min'),
                      style={'width': '10%', 'display': 'inline-block'}),
                  html.Div(
                      dcc.Dropdown(options=[{"label":str(i),"value":i} for i in range(0,1501)], 
                                   searchable=True, 
                                   placeholder='Max age weeks', 
                                   id='range-max'),
                      style={'width': '10%', 'display': 'inline-block'}),
                 ]

# Input fields for advanced options
crud_input = [html.Div([
                   html.Div([
                           html.Div([
                                  html.Div(
                                      dcc.Dropdown(df.animal_type.unique(), searchable=True, placeholder='Animal type', id='update_type'), 
                                      style={'width': '50%', 'display': 'inline-block'}),
                                  html.Div(
                                      dcc.Dropdown(df.breed.unique(), searchable=True, placeholder='Breed', id='update_breed'), 
                                      style={'width': '50%', 'display': 'inline-block'}),
                               ], 
                            ),
                           html.Div([
                                  html.Div(
                                      dcc.Dropdown(df.color.unique(), searchable=True, placeholder='Color', id='update_color'), 
                                      style={'width': '50%', 'display': 'inline-block'}),
                                  html.Div(
                                      dcc.Dropdown(df.outcome_subtype.unique(), searchable=True, placeholder='Outcome subtype', id='update_subtype'), 
                                      style={'width': '50%', 'display': 'inline-block'}),
                               ], 
                            ),
                           html.Div([
                                  html.Div(
                                      dcc.Dropdown(df.outcome_type.unique(), searchable=True, placeholder='Outcome type', id='update_outcome_type'), 
                                      style={'width': '50%', 'display': 'inline-block'}),
                                  html.Div(
                                      dcc.Dropdown(df.sex_upon_outcome.unique(), searchable=True, placeholder='Sex', id='update_sex'), 
                                      style={'width': '50%', 'display': 'inline-block'}),
                               ], 
                           ),
                           html.Div([
                               html.Div(
                                    html.Button(id='create_btn', children="Add New Animal", style=css.button('30px', '150px')),
                                    style={'display': 'none'},
                                    id='show_create_btn',
                                ),
                               html.Div(
                                   html.Button(id='update_btn', children="Update Animal", style=css.button('30px', '150px')),
                                   style={'display': 'none'},
                                   id='show_update_btn',
                               ),
                               html.Div(
                                   html.Button(id='input_reset_btn', children="Reset Input", n_clicks=0, style=css.button('30px', '100px')),
                                   style={'display': 'inline-block'},
                                   id='show_input_reset_btn',
                               ),
                               ]
                           ),
                           html.Div(id='crud_result'),
                        ],
                        style={'width': '30%', 'display': 'inline-block'},
                ),
               html.Div([
                        html.Div(dcc.Input(id='update_name', type="text", placeholder='Name', value='', style=css.input('34px', '150px'))),
                        html.Div(dcc.DatePickerSingle(id='update_dob', placeholder='DOB', date=None)),
                        ],
                        style={'display': 'inline-block'},
                       ),
               html.Div([
                   html.Div(
                       dcc.Input(id="address".format("text"), type="text", placeholder='Adress', value='', style=css.input('30px', '200px'))
                   ),
                   html.Div([
                            html.Div(
                                dcc.Input(id="city".format("text"), type="text", placeholder='City', value='', style=css.input('30px', '145px')),
                                style={'display': 'inline-block'},
                            ),
                           html.Div(
                               dcc.Input(id="state".format("text"), type="text", placeholder='State', value='', style=css.input('30px', '50px')),
                               style={'display':'inline-block'},
                           ),
                           ]
                    ),
                   html.Div(dcc.Input(id="zip".format("text"), type="text", placeholder='Zip', value='', style=css.input('30px', '75x'))),
                   html.Button('Enter Address', id='check_address', n_clicks=0, style=css.button('30px', '150px')),
                   ],
                    style={'width': '20%', 'display': 'inline-block'}
                ),
               html.Div([
                        html.Div(id='update_map'),
                        html.Div(id='lat_lon'),
                       ],
                        style={'display': 'inline-block'},
                ),
               ],
                style={'display': 'flex', 'align': 'top'}
            )
        ]


##########################
# Dashboard Layout / View
##########################

app = Dash(__name__)

# Load Grazioso Salvare logo
image_filename = 'GraziosoSalvareLogo1.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.config.suppress_callback_exceptions = True

# Dashboard layout
app.layout = html.Div([
    
    # Logo with link to Grazioso Salvare homepage
    html.A(
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
        href='https://www.snhu.edu'),
    
    # Title and unique identifier
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard'))),
    html.Center(html.H3('Created By: David France')),
    html.Center([html.Div([
                        html.Div(id='logged_in_user'),
                        html.Div(html.Button('Logout', id='logout_val', n_clicks=0, style=css.button('30px', '100px')), 
                                 style={'display': 'none'}, id='show_logout_btn'),
                        ],
                    )
                ],
               ),
    html.Hr(),

    # Login/Logout components
    html.Center([
        html.Div(
            id='show_login_fields',
            children=[
                html.Div([dcc.Input(
                            id="input_user".format("text"),
                            type="text",
                            placeholder="Username",
                            style=css.input('30px', '150px')
                        ),
                       dcc.Input(
                            id="input_passwd".format("password"),
                            type="password",
                            placeholder="Password",
                            style=css.input('30px', '150px')
                       ),
                      ]
                     ),
                html.Div(html.Button('Submit', id='submit-val', n_clicks=0, style=css.button('30px', '100px')))
            ]
        ),
        
            ]
           ),
    
    # Rescue animal radio buttons with reset button
    html.Div(id='dashboard',
             style={'display': 'none'},
             children=[
                 html.Center(html.Div([
                                    html.H3('Rescue Type'),
                                    dcc.RadioItems([{'label':'Water  ','value':'water'},
                                                    {'label':'Mountain/Wilderness  ', 'value':'mountain_wilderness'}, 
                                                    {'label':'Disaster/Individual Tracking', 'value':'disaster_tracking'}],
                                                    value=False,
                                                    inline=True,
                                                    id='radio_btn',
                                                    style={'font-size': '20px'}),
                                    ]
                                     )
                            ),
        html.Hr(),
        
        # Dropdown filters
        html.Div(dropdown_filters, id='dropdown_filters'),
        html.Center(html.Div(html.Button(id='reset_btn', n_clicks=0, children='Reset Filters', style=css.button('30px', '175px')))),               
        html.Hr(),
        
        # Advanced options
        html.Div([
                html.Div(html.Button(id='show_advanced_options', children='Show Advanced Options', n_clicks=0, style=css.button('25px', '200px')),
                         style={'display': 'inline-block'}
                        ),
                html.Div(
                        id='error_message',
                        style={'display': 'inline-block'},
                        ),
                ]
        ),
        # Div to hold all options available if user has read/write permission
        html.Div(id='advanced_options',
                 children=[# Radio buttons to select which advanced option the user wants
                            html.Div(dcc.RadioItems([{'label':'Add Animal','value':'create'},
                                                     {'label':'Update Animal', 'value':'update'}, 
                                                     {'label':'Delete Animal', 'value':'delete'}],
                                                    value=False,
                                                    inline=True,
                                                    id='advanced_radio_btn',
                                                    style={'font-size': '18px'}
                                                   )
                                    ),
                            # Each button is only shown when respective radio button is selected
                           html.Div(dcc.Input(id='update_id', type="text", placeholder='Animal ID', value='', style=css.input('30px', '150px')),
                                    style={'display': 'none'},
                                    id='show_update_id',
                                   ),
                html.Div(
                           dcc.ConfirmDialogProvider(
                               children=html.Div(html.Button(id='delete_btn', children="Delete Animal", style=css.button('30px', '150px'))),
                               id='delete_confirm',
                               message='Are you sure you want to delete this animal from the database?',
                           ),
                           style={'display': 'none'},
                           id='show_delete_btn', 
                       ),
                html.Div(id='delete_result', style={'display': 'none'}),
                           
               # Create/update data input
               html.Div(crud_input,
                      style={'display': 'none'},
                      id='advanced_filters'
                       ),
               
                        ],
                     style={'display': 'none'}
                ),   
        # Allows resetting advanced input without hiding create/update button 
        # Global variables are not recommended for Dash Apps
        html.Div(children='', id='create_update_value', style={'display': 'none'}),
        html.Hr(),
        
        # Set up area for data table
        html.Div(id='datatable_div',),
        html.Br(),
        html.Hr(),
        
        # Set up areas for donut chart of breeds on the lower left and 
        # geolocation map on lower right of dashboard
        html.Div(className='row',
            style={'display' : 'flex'},
            children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
            ),
            html.Div(
                id='map-id',
                className='col s12 m6',
            )
            ])
        ]),
    ],
    style={'background-color': '#EEEFF2',
          'font-family': 'Calibri'}
)



######################
# Component Callbacks
######################

# Login 'submit' button callback - populates data table if login successful
@app.callback(
    Output('datatable_div', 'children'),
    Output('logged_in_user', 'children'),
    Output('show_logout_btn', 'style'),
    Output('show_login_fields', 'style'),
    Output('dashboard', 'style'),
    Output('submit-val', 'n_clicks'),
    Output('animal_select', 'value'),
    Input('submit-val', 'n_clicks'),
    Input('input_user', 'value'),
    Input('input_passwd', 'value'),
    prevent_initial_call=True
)
def login(n_clicks, username, password):
    show_login_btn = {'display': 'none'}
    show_login_fields = {'display': 'block'}
    dashboard = {'display': 'none'}
    if username is None or password is None:
        return '', '', show_login_btn, show_login_fields, dashboard, 0, ''
    if not n_clicks:
        return '', '', show_login_btn, show_login_fields, dashboard, 0, ''
        
    if db_users.check_user(username, password):
        # Start with data of just Dogs to improve initial load time
        df = pd.DataFrame.from_records(db.find({'animal_type': 'Dog'}))
        logged_in_user = 'You are logged in as ' + username

        # Create data table
        data_table = dash_table.DataTable(
            id='datatable-id',
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
            ],
            data=df.to_dict('records'),
            row_selectable = 'single', # Allow selection of one row at a time
            selected_rows = [0],       # Default select first row
            page_size = 10,            # Set results per page to 10
            page_current = 0,          # Set first page as default
            sort_action = 'native',    # Allow sorting of columns
            filter_action = 'native'   # Allow filtering
            )
        animal_type = ['Dog']
        show_login_btn = {'display': 'block'}
        show_login_fields = {'display': 'none'}
        dashboard = {'display': 'block'}
    else:
        logged_in_user = 'Incorrect username or password'
        data_table = ''
        animal_type = ''

    return data_table, logged_in_user, show_login_btn, show_login_fields, dashboard, 0, animal_type

# Logout button callback
@app.callback(
    Output('show_login_fields', 'style', allow_duplicate=True),
    Output('show_logout_btn', 'style', allow_duplicate=True),
    Output('dashboard', 'style', allow_duplicate=True),
    Output('logged_in_user', 'children', allow_duplicate=True),
    Output('datatable_div', 'children', allow_duplicate=True),
    Output('logout_val', 'n_clicks'),
    Input('logout_val', 'n_clicks'),
    prevent_initial_call=True
)
def logout(n_clicks):
    if n_clicks:
        return {'display': 'block'}, {'display': 'none'}, {'display': 'none'}, 'Logged out', 'Logged out', 0
    
# Radio button selection callback to handle rescue animal queries  
@app.callback(
            Output('datatable-id', 'data', allow_duplicate=True),
            Output('datatable-id','selected_rows', allow_duplicate=True),
            Output('datatable-id','page_current', allow_duplicate=True),
            Input('radio_btn', 'value'),
            prevent_initial_call=True
            )
def rescue_radio_button(filter_type):
    if not filter_type:   
        df = pd.DataFrame.from_records(db.find({})) 
    # Water type constraints
    elif filter_type == 'water':
        df = pd.DataFrame.from_records(db.find({'rescue_type': 'water',
                                                'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}))   
    # Mountain/wilderness type constraints
    elif filter_type == 'mountain_wilderness':
        df = pd.DataFrame.from_records(db.find({'rescue_type': 'mountain_wilderness',
                                                'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}})) 
    # Disaster/individual tracking type constraints
    elif filter_type == 'disaster_tracking':
        df = pd.DataFrame.from_records(db.find({'rescue_type': 'disaster_tracking',
                                                'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}})) 

    return df.to_dict('records'), [0], 0

# Filters callback  
@app.callback(
            Output('datatable-id', 'data', allow_duplicate=True),
            Output('datatable-id','selected_rows', allow_duplicate=True),
            Output('datatable-id','page_current', allow_duplicate=True),
            Input('animal_select', 'value'),
            Input('breed_select', 'value'),
            Input('color_select', 'value'),
            Input('sex_select', 'value'),
            Input('range-min', 'value'),
            Input('range-max', 'value'),
            prevent_initial_call=True
            )
def filters(animal, breed, color, sex, min, max):
    search_dict = __filter_dict(animal, breed, color, sex, min, max)     
    df = pd.DataFrame.from_records(db.find(search_dict)) 

    return df.to_dict('records'), [0], 0

# Reset button callback unselects all radio buttons
@app.callback(
            Output('radio_btn','value', allow_duplicate=True),
            Output('dropdown_filters', 'children'),
            Input('reset_btn', 'n_clicks'),
            prevent_initial_call=True
            )
def filter_reset_button(n_clicks):
    if n_clicks:
        return False, dropdown_filters

# 'Advanced Options' callback displays create/update/delete radio options if user has read/write permission
@app.callback(
    Output('advanced_options', 'style'),
    Output('show_advanced_options', 'children'),
    Output('error_message', 'children'),
    Output('show_advanced_options', 'n_clicks'),
    Input('show_advanced_options', 'n_clicks'),
    Input('input_user', 'value'),
    Input('input_passwd', 'value'),
    prevent_initial_call=True
)
def advanced_options(n_clicks, user, password):
    if user is None or password is None:
        return {'display': 'none'}, 'Show Advanced Options', '', 0
    if n_clicks:
        if not db_users.check_permission(user, password):
            return {'display': 'none'}, 'Show Advanced Options', "You don't have permission", 0
        elif n_clicks % 2 == 1:
            return {'display': 'block'}, 'Hide Advanced Options', '', n_clicks
            
    return {'display': 'none'}, 'Show Advanced Options', '', 0

# Advanced options radio button callback - only display options for selected function
@app.callback(
    Output('show_create_btn', 'style', allow_duplicate=True),
    Output('show_update_btn', 'style', allow_duplicate=True),
    Output('show_delete_btn', 'style'),
    Output('show_update_id', 'style'),
    Output('advanced_filters', 'style'),
    Output('delete_result', 'style'),
    Output('create_update_value', 'children'),
    Input('advanced_radio_btn', 'value'),
    prevent_initial_call=True
)
def advanced_radio_btn(value):
    # Set defaults to display nothing
    create = {'display': 'none'}
    update = {'display': 'none'}
    delete = {'display': 'none'}
    id_input = {'display': 'none'}
    update_id_input = {'display': 'none'}
    input = {'display': 'none'}
    delete_result = {'display': 'none'}
    create_update=''

    # Display corresponding fields for each radio button
    if value == 'create':
        create = {'display': 'inline-block'}
        input = {'display': 'block'}
        create_update = 'create'
    elif value == 'update':
        update = {'display': 'inline-block'}
        id_input = {'width': '10%', 'display': 'inline-block'}
        input = {'display': 'block'}
        create_update = 'update'
    elif value == 'delete':
        delete = {'display': 'block'}
        delete_result = {'display': 'block'}
        id_input = {'width': '10%', 'display': 'inline-block'}

    return create, update, delete, id_input, input, delete_result, create_update

# Delete animal callack
@app.callback(
    Output('delete_result', 'children', allow_duplicate=True),
    Output('dropdown_filters', 'children', allow_duplicate=True),
    Output('radio_btn','value', allow_duplicate=True),
    Input('delete_confirm', 'submit_n_clicks'),
    Input('update_id', 'value'),
    prevent_initial_call=True
)
def delete_button(submit_n_clicks, animal_id):
    if ctx.triggered_id == 'update_id':
        raise PreventUpdate
    if not submit_n_clicks:
        raise PreventUpdate
    if animal_id is None or animal_id == '':
        return "No animal id specified", dropdown_filters, False
        
    # Get entire animal record from database - not all data is loaded into data table
    # find() returns an array, animal dictionary is first element
    result = db.find({'animal_id': animal_id})

    if len(result) == 0:
        return f"Animal - {animal_id} does not exist", dropdown_filters, False

    animal = result[0]
    
    # Insert animal into backup database before deleting
    result = db_backup.insert_backup(animal)
    if result:
        deleted_count = db.delete({'animal_id': animal_id})  
        if deleted_count:
            result_message = "Deleted animal ID - " + animal['animal_id']
        else:
            result_message = "FAILED- animal not deleted"
    else:
        result_message = "FAILED- animal not deleted"
        
    return result_message, dropdown_filters, False                       

# Create new animal callback
@app.callback(
    Output('crud_result', 'children', allow_duplicate=True),
    Output('create_btn', 'n_clicks'),
    Output('dropdown_filters', 'children', allow_duplicate=True),
    Output('radio_btn','value', allow_duplicate=True),
    Input('create_btn', 'n_clicks'),
    Input('update_type', 'value'),
    Input('update_breed', 'value'),
    Input('update_color', 'value'),
    Input('update_dob', 'date'),
    Input('update_name', 'value'),
    Input('update_subtype', 'value'),
    Input('update_outcome_type', 'value'),
    Input('update_sex', 'value'),
    Input('lat_lon', 'children'),
    prevent_initial_call=True
)
def create_animal(n_clicks, type, breed, color, dob, name, subtype, outcome_type, sex, location):
    if not n_clicks:
        raise PreventUpdate

    if not __check_date(dob):
        return 'Invalid date', 0, dropdown_filters, False
        
    # Create dictionary of column names and values
    dict =  __animal_dict(type, breed, color, dob, name, subtype, outcome_type, sex, location)
    result = db.insert(dict)
    
    return result, 0, dropdown_filters, False
    
# Update animal callback
@app.callback(
    Output('crud_result', 'children', allow_duplicate=True),
    Output('update_btn', 'n_clicks'),
    Output('dropdown_filters', 'children', allow_duplicate=True),
    Output('radio_btn','value', allow_duplicate=True),
    Input('update_btn', 'n_clicks'),
    Input('update_id', 'value'),
    Input('update_type', 'value'),
    Input('update_breed', 'value'),
    Input('update_color', 'value'),
    Input('update_dob', 'date'),
    Input('update_name', 'value'),
    Input('update_subtype', 'value'),
    Input('update_outcome_type', 'value'),
    Input('update_sex', 'value'),
    Input('lat_lon', 'children'),
    prevent_initial_call=True
)
def update_animal(n_clicks, id, type, breed, color, dob, name, subtype, outcome_type, sex, location):
    if not n_clicks:
        raise PreventUpdate

    # ID is required at this time
    if not id or not db.check_id(id):
        return 'Valid animal ID to update is required', 0, dropdown_filters, False
        
    # Create dictionary of all inputted values to be changed
    dict =  __animal_dict(type, breed, color, dob, name, subtype, outcome_type, sex, location)
    result = db.update({'animal_id': id}, dict)
    if result:
        message = f"Animal with ID {id} successfully updated"
    else:
        message = f"Failed to update animal with ID {id}"

    return message, 0, dropdown_filters, False

# Reset input callback
@app.callback(
    Output('advanced_filters', 'children', allow_duplicate=True),
    Output('input_reset_btn', 'n_clicks'),
    Output('show_create_btn', 'style', allow_duplicate=True),
    Output('show_update_btn', 'style', allow_duplicate=True),
    Input('input_reset_btn',  'n_clicks'),
    Input('create_update_value', 'children'),
    prevent_initial_call=True
)
def advanced_input_reset(n_clicks, create_update):
    if not n_clicks:
        raise PreventUpdate
  
    create_button = {'display': 'none'}
    update_button = {'display': 'none'}
    if create_update == 'create':
        create_button = {'display': 'inline-block'}
    elif create_update == 'update':
        update_button = {'display': 'inline-block'}
    
    return crud_input, 0, create_button, update_button
    
# Highlight a row on the data table when the user selects it
@app.callback(
        Output('datatable-id', 'style_data_conditional'),
        Input('datatable-id', 'selected_rows'),
        prevent_initial_call=True
        )
def highlight_row(selected_row):
    return [{
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selected_row]  
    
# Donut chart to display the breeds of animal based on quantity represented in
# the data table
@app.callback(
        Output('graph-id', 'children'),
        Input('datatable-id', 'derived_virtual_data'),
        prevent_initial_call=True
        )
def update_graphs(view_data):
    if view_data is None:
        return
    
    dff = pd.DataFrame.from_dict(view_data)
    if len(dff.columns) == 0:
        return
        
    load_figure_template('pulse')
    fig = px.pie(dff, names='breed', 
                 title='Preferred Animals', 
                 color_discrete_sequence=px.colors.qualitative.Prism,
                 template='pulse',
                 hole=0.4)
    fig.update_traces(textposition='inside')
    fig.update_layout(uniformtext_minsize=12, 
                      uniformtext_mode='hide', 
                      title_x=0.5, 
                      title_font_size = 25)
    return dcc.Graph(figure=fig)

# Update marker on map to reflect row selected
@app.callback(
        Output('map-id', 'children'),
        Input('datatable-id', 'derived_virtual_data'),
        Input('datatable-id', 'derived_virtual_selected_rows'),
        prevent_initial_call=True
        )
def update_map(view_data, index):
    if view_data is None:
        return
    elif index is None or len(index) < 1:
        return
    
    dff = pd.DataFrame.from_dict(view_data)
    if len(dff.columns) == 0:
        return
       
    row = index[0]
    
    # Map is centered on Austin TX at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[30.75,-97.48], zoom=8, children=[
           dl.TileLayer(id="base-layer-id"),
               
           # Marker with tool tip and popup
           dl.Marker(position=[dff.iloc[row,11],dff.iloc[row,12]], # Column 11 and 12 define grid coordinates
              children=[
              dl.Tooltip(dff.iloc[row,4]), # Col 4 is breed of animal
              dl.Popup([
                 html.H2("Animal Name"),
                 html.H3(dff.iloc[row,7])  # Col 9 is name of animal
             ])
          ])
       ])
    ]

# Update map for animal update/insert location in 'Advanced Options'
@app.callback(
        Output('update_map', 'children'),
        Output('lat_lon', 'children'),
        Output('check_address', 'n_clicks'),
        Input('check_address', 'n_clicks'),
        Input('address', 'value'),
        Input('city', 'value'),
        Input('state', 'value'),
        Input('zip', 'value'),
        prevent_initial_call=True
        )
def address_map(n_clicks, street, city, state, zip):
    if not n_clicks:
        return '', '', 0

    # Nominatim returns locatation data from given address
    geoloc = Nominatim(user_agent='AAC_Dashboard')
    address = f'{street}, {city}, {state}, {zip}'

    # If address doesn't exist, return error message
    try:
        location = geoloc.geocode(address).raw
    except:
        return f"{address} is a Bad Address", '', 0

    # Pull latitude and longitude values from location data, use map to show user where they've selected
    lat_lon = [location['lat'], location['lon']]
    return dl.Map(style={'width': '400px', 'height': '300px'},
                  center=lat_lon, 
                  zoom=14,
                  children=[
                      dl.TileLayer(id="base-layer-id",
                                   attribution='Map Data &copy; <a href="https://www.openstreetmap.org/">OpenStreetMap</a>'
                                  ),
                      dl.Marker(position=lat_lon, 
                                children=[dl.Tooltip(address)
                                         ]
                               )
                  ]), lat_lon, 0
    
# Convert filter values to JSON dictionary to be passed to MongoDB database
def __filter_dict(animal, breed, color, sex, min, max):
    search_dict = {}
    
    if animal is not None and len(animal) > 0:
        search_dict['animal_type'] = {'$in': animal}

    if breed is not None and len(breed) > 0:
        search_dict['breed'] = {'$in': breed}

    if color is not None and len(color) > 0:
        search_dict['color'] = {'$in': color}

    if sex is not None and len(sex) > 0:
        search_dict['sex_upon_outcome'] = {'$in': sex}

    if min is not None:
        min_age = min
    else:
        min_age = 0

    if max is not None:
        max_age = max
    else:
        max_age = 1600
    
    search_dict['age_upon_outcome_in_weeks'] = {'$gte': min_age, '$lte': max_age}

    return search_dict

# Creates dictionary of category names and provided values
def __animal_dict(type, breed, color, dob, name, subtype, outcome_type, sex, location):
    animal_dict = {}

    if type is not None:
        animal_dict['animal_type'] = type

    if breed is not None:
        animal_dict['breed'] = breed

    if color is not None:
        animal_dict['color'] = color

    if dob is not None:
        animal_dict['date_of_birth'] = dob

    if name is not None and name != '' and db.name_validation(name):  # name is the only parameter needing additional validation
        animal_dict['name'] = name

    if subtype is not None:
        animal_dict['outcome_subtype'] = subtype

    if outcome_type is not None:
        animal_dict['outcome_type'] = outcome_type

    if sex is not None:
        animal_dict['sex_upon_outcome'] = sex

    if location is not None:
        animal_dict['location_lat'] = location[0]
        animal_dict['location_long'] = location[1]

    return animal_dict

# Date input validation
def __check_date(date):
    try:
        datetime.strptime(date, "%Y-%m-%d")
        return True
    except:
        return False
    
        
app.run(jupyter_mode="external", debug=True)


Dash app running on http://127.0.0.1:8050/
---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
Cell In[17], line 832, in __animal_dict(
    type='Other',
    breed='Bat',
    color='Blue/White',
    dob='2023-02-03',
    name='Sugma',
    subtype='Foster',
    outcome_type='Adoption',
    sex='Intact Male',
    location=''
)
    829     animal_dict['sex_upon_outcome'] = sex
    831 if location is not None:
--> 832     animal_dict['location_lat'] = location[0]
        animal_dict = {'animal_type': 'Other', 'breed': 'Bat', 'color': 'Blue/White', 'date_of_birth': '2023-02-03', 'name': 'Sugma', 'outcome_subtype': 'Foster', 'outcome_type': 'Adoption', 'sex_upon_outcome': 'Intact Male'}
        location = ''
    833     animal_dict['location_long'] = location[1]
    835 return animal_dict

IndexError: string index out of range

------------------------------------------------------------------